In [ ]:
import logging
from pathlib import Path
from Pegasus.api import *
import glob
import os
logging.basicConfig(level = logging.DEBUG)

# import os
# os.environ['KAGGLE_USERNAME'] = "vedula"
# os.environ['KAGGLE_KEY'] = "482a5c14ced45f63f3698eacb8fa0c62"

# import kaggle
# kaggle.api.dataset_download_files('nikhilpandey360/chest-xray-masks-and-labels/download', path='.', unzip=True)

props = Properties()
props["pegasus.dir.storage.mapper"] = "Replica"
props["pegasus.dir.storage.mapper.replica"] = "YAML"
props["pegasus.dir.storage.mapper.replica.file"] = "output-rc.yml"
props.write() 

preprocess = Transformation(
                "preprocess",
                site="local",
                pfn="/home/scitech/shared-data/mask_wf/preprocess.py",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

train_model = Transformation( 
                "train_model",
                site="local",
                pfn="/home/scitech/shared-data/mask_wf/train_model.py",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

# test_model = Transformation( 
#                 "test_model",
#                 site="local",
#                 pfn="/home/scitech/shared-data/mask_wf/test.py",
#                 is_stageable=True,
#                 arch=Arch.X86_64,
#                 os_type=OS.LINUX
#             )

tc = TransformationCatalog().add_transformations(preprocess).write()

file_list = []
output_list = []

rc = ReplicaCatalog()

for file in glob.glob("./train_images/*.png"):
    f = file.replace("./train_images/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./train_images/").resolve() / f)
    
for file in glob.glob("./train_masks/*.png"):
    f = file.replace("./train_masks/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./train_masks/").resolve() / f)
    
rc.write()

for filename in glob.glob("./train_images/*.png"):
    f = filename.replace("./train_images/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))

for filename in glob.glob("./train_masks/*.png"):
    f = filename.replace("./train_masks/", '').strip(".png")+"_norm.png"
    output_list.append(File(f))
    
output_rc = ReplicaCatalog()\
                .add_replica("local", "f1", Path(".").resolve() / "wf-output/norm_images/")\
                .add_replica("local", "f2", Path(".").resolve() / "wf-output/norm_masks/")\
                .write("output-rc.yml")
    
        
wf = Workflow("preprocess")
    
job_preprocess = Job(preprocess)\
                    .add_inputs(*file_list)\
                    .add_outputs(*output_list)

file_list.clear()

for file in glob.glob("./norm_images/*.png"):
    f = file.replace("./norm_images/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./norm_images/").resolve() / f)
    
for file in glob.glob("./norm_masks/*.png"):
    f = file.replace("./norm_masks/", '')
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path("./norm_masks/").resolve() / f)
    
fb = File("saved_model.pb")


job_train = Job(train_model)\
                    .add_inputs(*file_list)\
                    .add_outputs(fb)


wf.add_jobs(job_preprocess)

try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)  

Plan:
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.08.25 17:25:35.184 UTC:    
2020.08.25 17:25:35.189 UTC:   ----------------------------------------------------------------------- 
2020.08.25 17:25:35.195 UTC:   File for submitting this DAG to HTCondor           : preprocess-0.dag.condor.sub 
2020.08.25 17:25:35.200 UTC:   Log of DAGMan debugging messages                 : preprocess-0.dag.dagman.out 
2020.08.25 17:25:35.206 UTC:   Log of HTCondor library output                     :

[#################---------------------------------]  33.3% ..Running (Completed: 3, Queued: 0, Running: 1, Failed: 0)